To analyze results from tfevents file

In [1]:
import sys
sys.path.append("./../")

In [2]:
import os
import json
import glob

import numpy as np
import pandas as pd

from tbparse import SummaryReader
import matplotlib.pyplot as plt
plt.style.use('seaborn')

2022-07-30 11:27:50.387143: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-30 11:27:50.387243: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
def extract_results(model_dir):
    """
        Get metrics from a give model folder
    """
    
    # Get config
    config_json = os.path.join(model_dir, 'config.json')
    config = json.load(open(config_json, 'r'))
    
    # Get test events file
    tblog_dir = os.path.join(model_dir, "tblog")
    events_file = glob.glob(tblog_dir + "/events.out.tfevents*.1") # *.1 will be from test
    if len(events_file) > 0:
        events_file = events_file[0]
    else:
        print("Skipping", model_dir)
        return None
    tb_log = SummaryReader(events_file, pivot=True).scalars
    
    # Extract key points
    lam_sl = config['method_params']['lam_sl']
    ds_size = config['ds_params']['size']
    ds_corr = config['ds_params']['corruption']
    acc = tb_log.test_acc.values[0]
    ece = tb_log.test_ece.values[0]
    f1 = tb_log.test_f1.values[0]
    loss = tb_log.test_loss.values[0]
    
    return {
        'lam_sl': lam_sl,
        'ds_size': ds_size,
        'ds_corr': ds_corr,
        'acc': acc,
        'ece': ece,
        'f1': f1,
        'loss': loss
    }

In [4]:
def extract_best_results(model_dir):
    # Get config
    config_json = os.path.join(model_dir, 'config.json')
    config = json.load(open(config_json, 'r'))
    
    # Get test events file
    tblog_dir = os.path.join(model_dir, "tblog")
    events_file = glob.glob(tblog_dir + "/events.out.tfevents*.0") # *.1 will be from test
    if len(events_file) > 0:
        events_file = events_file[0]
    else:
        print("Skipping", model_dir)
        return None
    tb_log = SummaryReader(events_file, pivot=True).scalars
    
    # Get the best point in term of val_f1 (dont look at ECE)
    try:
        tb_log = tb_log[tb_log.val_f1 == tb_log.val_f1.max()]
    except:
        print(model_dir)
        return None
    
    # Extract key points
    lam_sl = config['method_params'].get('lam_sl', 0.0)
    ds_size = config['ds_params'].get('size', 'Full')
    ds_corr = config['ds_params'].get('corruption', 'identity')
    acc = tb_log.val_acc.values[0]
    ece = tb_log.val_ece.values[0]
    f1 = tb_log.val_f1.values[0]
    loss = tb_log.val_loss.values[0]
    
    return {
        'lam_sl': lam_sl,
        'ds_size': ds_size,
        'ds_corr': ds_corr,
        'acc': acc,
        'ece': ece,
        'f1': f1,
        'loss': loss
    }

In [26]:
# rootdir = "./../zoo/bmnist53-mfvi/"
# model_str = "ConvNet"

# rootdir = "./../zoo/abl-alpha100-cifar10im-mfvi/"
# model_str = "VGG11"

rootdir = "./../zoo/abl-alpha100-unibin/"
model_str = "VGG11"

# rootdir = "./../zoo/abl-alpha100-slim-CIFAR10/"
# model_str = "VGG11"

In [27]:
modeldirs = []
for expdir in map(lambda x: os.path.join(rootdir, x), os.listdir(rootdir)):
    _modeldirs = os.listdir(os.path.join(expdir, model_str))
    modeldirs.extend(map(lambda x: os.path.join(expdir, model_str, x), _modeldirs))

In [28]:
modeldirs

['./../zoo/abl-alpha100-unibin/CIFAR10/VGG11/sl-lam1e-02-1-20220430031115',
 './../zoo/abl-alpha100-unibin/CIFAR10/VGG11/sl-lam1e-04-3-20220430001406',
 './../zoo/abl-alpha100-unibin/CIFAR10/VGG11/sl-lam1e-01-4-20220430043957',
 './../zoo/abl-alpha100-unibin/CIFAR10/VGG11/sl-lam1e-02-3-20220430031115',
 './../zoo/abl-alpha100-unibin/CIFAR10/VGG11/sl-lam1e+00-2-20220430060836',
 './../zoo/abl-alpha100-unibin/CIFAR10/VGG11/sl-lam1e-02-2-20220430031115',
 './../zoo/abl-alpha100-unibin/CIFAR10/VGG11/sl-lam1e-03-3-20220430014227',
 './../zoo/abl-alpha100-unibin/CIFAR10/VGG11/sl-lam1e-05-3-20220429224537',
 './../zoo/abl-alpha100-unibin/CIFAR10/VGG11/sl-lam1e+00-1-20220430060829',
 './../zoo/abl-alpha100-unibin/CIFAR10/VGG11/sl-lam1e-04-5-20220430001339',
 './../zoo/abl-alpha100-unibin/CIFAR10/VGG11/sl-lam1e-03-1-20220430014218',
 './../zoo/abl-alpha100-unibin/CIFAR10/VGG11/sl-lam1e-01-1-20220430043957',
 './../zoo/abl-alpha100-unibin/CIFAR10/VGG11/sl-lam1e-03-4-20220430014218',
 './../zoo/a

In [29]:
# df_results = pd.DataFrame(filter(None, [extract_results(mdir) for mdir in modeldirs]))
df_results = pd.DataFrame(filter(None, [extract_best_results(mdir) for mdir in modeldirs]))

In [30]:
df_results

,lam_sl,ds_size,ds_corr,acc,ece,f1,loss
0,0.010000,Full,identity,0.7885,0.173961,0.7885,1.148332
1,0.000100,Full,identity,0.8115,0.040267,0.8115,0.801930
2,0.100000,Full,identity,0.5399,0.126023,0.5399,3.125296
3,0.010000,Full,identity,0.7915,0.155241,0.7915,1.179996
4,1.000000,Full,identity,0.1277,0.135602,0.1277,70.262367
5,0.010000,Full,identity,0.7892,0.150034,0.7892,0.848587
6,0.001000,Full,identity,0.8066,0.131840,0.8066,0.704781
7,0.000010,Full,identity,0.8059,0.053080,0.8059,0.875734
8,1.000000,Full,identity,0.2154,0.058706,0.2154,80.621628
9,0.000100,Full,identity,0.8084,0.038865,0.8084,0.776761


In [31]:
agg_results = {}

gdf_corr = df_results.groupby('ds_corr')
for corr_str, gdf1 in gdf_corr:
    agg_results[corr_str] = {}
    gdf_ds = gdf1.groupby('ds_size')
    
    for ds_size, gdf2 in gdf_ds:
        gdf_lam = gdf2.groupby('lam_sl')
        
        r = []
        for lam_sl, _df in gdf_lam:
            n = np.sqrt(_df.shape[0])

            d = {
                'lam_sl': lam_sl,
                'acc_mean': _df.acc.mean(),
                'acc_err': _df.acc.std() / n,
                'ece_mean': _df.ece.mean(),
                'ece_err': _df.ece.std() / n,
                'f1_mean': _df.f1.mean(),
                'f1_err': _df.f1.std() / n,
                'loss_mean': _df.loss.mean(),
                'loss_err': _df.loss.std() / n,
            }
            
            r.append(d)
        agg_results[corr_str][ds_size] = pd.DataFrame(r)

In [32]:
# agg_results = []

# gdf = df_results.groupby('lam_sl')
# for lam_sl, _df in gdf:
#     n = np.sqrt(_df.shape[0])
    
#     d = {
#         'lam_sl': lam_sl,
#         'acc_mean': _df.acc.mean(),
#         'acc_err': _df.acc.std() / n,
#         'ece_mean': _df.ece.mean(),
#         'ece_err': _df.ece.std() / n,
#         'f1_mean': _df.f1.mean(),
#         'f1_err': _df.f1.std() / n,
#         'loss_mean': _df.loss.mean(),
#         'loss_err': _df.loss.std() / n,
#     }
    
#     agg_results.append(d)

# df_agg = pd.DataFrame(agg_results)

In [33]:
agg_results['identity']['Full']

,lam_sl,acc_mean,acc_err,ece_mean,ece_err,f1_mean,f1_err,loss_mean,loss_err
0,0.000001,0.80762,0.002250,0.049183,0.004078,0.80762,0.002250,0.778279,0.036834
1,0.000010,0.80752,0.003145,0.042773,0.006292,0.80752,0.003145,0.793863,0.048957
2,0.000100,0.80898,0.001096,0.034411,0.005454,0.80898,0.001096,0.778561,0.034199
3,0.001000,0.80372,0.002054,0.126210,0.006342,0.80372,0.002054,0.765416,0.022491
4,0.010000,0.78842,0.001467,0.153961,0.005464,0.78842,0.001467,0.987689,0.076855
5,0.100000,0.42362,0.084777,0.125369,0.024802,0.42362,0.084777,5.553658,0.991624
6,1.000000,0.16372,0.020485,0.127672,0.031223,0.16372,0.020485,82.753300,5.435072


In [24]:
agg_results['identity']['Full']

,lam_sl,acc_mean,acc_err,ece_mean,ece_err,f1_mean,f1_err,loss_mean,loss_err
0,0.000001,0.81172,0.002151,0.040019,0.005384,0.81172,0.002151,0.785432,0.029433
1,0.000010,0.80830,0.000467,0.041690,0.005485,0.80830,0.000467,0.795235,0.032294
2,0.000100,0.81108,0.002436,0.042727,0.006873,0.81108,0.002436,0.771035,0.039378
3,0.001000,0.81064,0.003257,0.039674,0.007196,0.81064,0.003257,0.756541,0.020271
4,0.010000,0.79414,0.010211,0.058291,0.022492,0.79414,0.010211,0.533702,0.038635
5,0.100000,0.66988,0.113455,0.072656,0.015245,0.66988,0.113455,-1.021775,0.262256
6,1.000000,0.17606,0.018261,0.034472,0.006554,0.17606,0.018261,-20.232239,0.870965


In [ ]:
agg_results['identity']['Full']